In [1]:
// Import the required dependencies
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, IndexToString, StringIndexer, VectorIndexer}

Intitializing Scala interpreter ...

Spark Web UI available at http://FireBall:4041
SparkContext available as 'sc' (version = 2.4.0, master = local[*], app id = local-1552503973906)
SparkSession available as 'spark'


import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, IndexToString, StringIndexer, VectorIndexer}


In [2]:
val titanic_dataset = spark.read.
                  option("header","true").
                  option("inferSchema","true").
                  option("mode", "DROPMALFORMED").
                  csv("file:///C:/Users/FireBall/Downloads/SCALA_SPARK/Dataset/Titanic dataset/train.csv")
                  //.na.drop()    don't perform it just after reading the dataset. 
                  //Preprocess and then when we are fitting the model on dataset, we should drop NA/NULL fields.

titanic_dataset: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int ... 10 more fields]


In [3]:
titanic_dataset.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [4]:
titanic_dataset.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [5]:
titanic_dataset.createOrReplaceTempView("titanic")

In [6]:
spark.sql("select distinct Embarked from titanic order by Embarked").show()

+--------+
|Embarked|
+--------+
|    null|
|       C|
|       Q|
|       S|
+--------+



In [7]:
val colNames = titanic_dataset.columns

colNames: Array[String] = Array(PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked)


In [8]:
titanic_dataset.head(1)

res4: Array[org.apache.spark.sql.Row] = Array([1,0,3,Braund, Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.25,null,S])


In [9]:
titanic_dataset.head(0)

res5: Array[org.apache.spark.sql.Row] = Array()


In [10]:
val firstRow = titanic_dataset.head(1)(0)

firstRow: org.apache.spark.sql.Row = [1,0,3,Braund, Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.25,null,S]


In [11]:
println("\n")
println("First Row Data")
for ( i <- 1 until colNames.length){
    println(colNames(i) + " :\t" +firstRow(i))
}



First Row Data
Survived :	0
Pclass :	3
Name :	Braund, Mr. Owen Harris
Sex :	male
Age :	22.0
SibSp :	1
Parch :	0
Ticket :	A/5 21171
Fare :	7.25
Cabin :	null
Embarked :	S


In [12]:
val LORdata_all = titanic_dataset.select(titanic_dataset("Survived").as("label"), 
                                         $"Pclass",$"Sex",$"Age",$"SibSp",$"Parch",$"Fare",$"Embarked")

LORdata_all: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]


In [13]:
val LOR_data = LORdata_all.na.drop()

LOR_data: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]


In [14]:
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.OneHotEncoderEstimator

import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.OneHotEncoderEstimator


In [15]:
//Converting Strings into numerical values
val genderIndexer = new StringIndexer()
                        .setInputCol("Sex")
                        .setOutputCol("SexIndexer")
val embarkIndexer = new StringIndexer()
                        .setInputCol("Embarked")
                        .setOutputCol("EmbarkedIndexer")

genderIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_e0ff14faad24
embarkIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_1e784b20d52d


In [16]:
//Convert Numerical values to One HotEncoder 0 or 1
val genderEncoder = new OneHotEncoderEstimator()
                        .setInputCols(Array("SexIndexer"))
                        .setOutputCols(Array("SexVec"))
val embarkEncoder = new OneHotEncoderEstimator()
                        .setInputCols(Array("EmbarkedIndexer"))
                        .setOutputCols(Array("EmbarkedVec"))

genderEncoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_a2edcb67cb1c
embarkEncoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_2038cd46d496


In [17]:
//Label & Features
val assembeler = new VectorAssembler().setInputCols(Array("Pclass", "SexVec", "Age", "SibSp", 
                                                          "Parch", "Fare", "EmbarkedVec"))
                                      .setOutputCol("features")

assembeler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_f981d49bc03c


In [18]:
val LORalgo = new LogisticRegression()
                 //.setMaxIter(10)
                 //.setRegParam(0)
                 //.setElasticNetParam(0)

val pipeline = new Pipeline().setStages(Array(genderIndexer,embarkIndexer,genderEncoder,embarkEncoder,assembeler,LORalgo))

LORalgo: org.apache.spark.ml.classification.LogisticRegression = logreg_14fe6cb75229
pipeline: org.apache.spark.ml.Pipeline = pipeline_103b4251f93b


In [19]:
val model = pipeline.fit(LOR_data)

2019-03-14 00:37:22 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-03-14 00:37:22 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


model: org.apache.spark.ml.PipelineModel = pipeline_103b4251f93b


In [20]:
val test_dataset = spark.read.
                  option("header","true").
                  option("inferSchema","true").
                  option("mode", "DROPMALFORMED").
                  csv("file:///C:/Users/FireBall/Downloads/SCALA_SPARK/Dataset/Titanic dataset/test.csv")
                  //.na.drop()
val Survived_dataset = spark.read.
                  option("header","true").
                  option("inferSchema","true").
                  option("mode", "DROPMALFORMED").
                  csv("file:///C:/Users/FireBall/Downloads/SCALA_SPARK/Dataset/Titanic dataset/gender_submission.csv")
                  //.na.drop()

test_dataset: org.apache.spark.sql.DataFrame = [PassengerId: int, Pclass: int ... 9 more fields]
Survived_dataset: org.apache.spark.sql.DataFrame = [PassengerId: int, Survived: int]


In [21]:
test_dataset.join(Survived_dataset, test_dataset("PassengerId") === Survived_dataset("PassengerId"),"inner").show(5)

+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+-----------+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|PassengerId|Survived|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+-----------+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|        892|       0|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0| null|       S|        893|       1|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875| null|       Q|        894|       0|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625| null|       S|        895|       0|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875| null|       S|        896|       1|
+-----------+------+--------------------+------+----+---

In [22]:
test_dataset.createOrReplaceTempView("test_table")
Survived_dataset.createOrReplaceTempView("Survived_table")

val joined_df = spark.sql( "select t.*,s.Survived 
                            from test_table t 
                            inner join Survived_table s 
                            on s.PassengerId = t.PassengerId")
joined_df.show(5)

+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+--------+
|PassengerId|Pclass|                Name|   Sex| Age|SibSp|Parch| Ticket|   Fare|Cabin|Embarked|Survived|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+--------+
|        892|     3|    Kelly, Mr. James|  male|34.5|    0|    0| 330911| 7.8292| null|       Q|       0|
|        893|     3|Wilkes, Mrs. Jame...|female|47.0|    1|    0| 363272|    7.0| null|       S|       1|
|        894|     2|Myles, Mr. Thomas...|  male|62.0|    0|    0| 240276| 9.6875| null|       Q|       0|
|        895|     3|    Wirz, Mr. Albert|  male|27.0|    0|    0| 315154| 8.6625| null|       S|       0|
|        896|     3|Hirvonen, Mrs. Al...|female|22.0|    1|    1|3101298|12.2875| null|       S|       1|
+-----------+------+--------------------+------+----+-----+-----+-------+-------+-----+--------+--------+
only showing top 5 rows



joined_df: org.apache.spark.sql.DataFrame = [PassengerId: int, Pclass: int ... 10 more fields]


In [23]:
joined_df.createOrReplaceTempView("Joined_testdata")
spark.sql("select * from Joined_testdata where PassengerId = 892 ").show()


+-----------+------+----------------+----+----+-----+-----+------+------+-----+--------+--------+
|PassengerId|Pclass|            Name| Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|Survived|
+-----------+------+----------------+----+----+-----+-----+------+------+-----+--------+--------+
|        892|     3|Kelly, Mr. James|male|34.5|    0|    0|330911|7.8292| null|       Q|       0|
+-----------+------+----------------+----+----+-----+-----+------+------+-----+--------+--------+



In [24]:
val LOR_Testdata_all = joined_df.select(joined_df("Survived").as("label"), 
                                         $"Pclass",$"Sex",$"Age",$"SibSp",$"Parch",$"Fare",$"Embarked")

LOR_Testdata_all: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]


In [25]:
val LOR_Testdata = LOR_Testdata_all.na.drop()

LOR_Testdata: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 6 more fields]


In [26]:
LOR_Testdata.show(5)

+-----+------+------+----+-----+-----+-------+--------+
|label|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+-----+------+------+----+-----+-----+-------+--------+
|    0|     3|  male|34.5|    0|    0| 7.8292|       Q|
|    1|     3|female|47.0|    1|    0|    7.0|       S|
|    0|     2|  male|62.0|    0|    0| 9.6875|       Q|
|    0|     3|  male|27.0|    0|    0| 8.6625|       S|
|    1|     3|female|22.0|    1|    1|12.2875|       S|
+-----+------+------+----+-----+-----+-------+--------+
only showing top 5 rows



In [27]:
val results = model.transform(LOR_Testdata)

results: org.apache.spark.sql.DataFrame = [label: int, Pclass: int ... 14 more fields]


In [28]:
results.show(2)

+-----+------+------+----+-----+-----+------+--------+----------+---------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|label|Pclass|   Sex| Age|SibSp|Parch|  Fare|Embarked|SexIndexer|EmbarkedIndexer|       SexVec|  EmbarkedVec|            features|       rawPrediction|         probability|prediction|
+-----+------+------+----+-----+-----+------+--------+----------+---------------+-------------+-------------+--------------------+--------------------+--------------------+----------+
|    0|     3|  male|34.5|    0|    0|7.8292|       Q|       0.0|            2.0|(1,[0],[1.0])|    (2,[],[])|(8,[0,1,2,5],[3.0...|[0.38677297509601...|[0.59550561797752...|       0.0|
|    1|     3|female|47.0|    1|    0|   7.0|       S|       1.0|            0.0|    (1,[],[])|(2,[0],[1.0])|[3.0,0.0,47.0,1.0...|[0.38677297509601...|[0.59550561797752...|       0.0|
+-----+------+------+----+-----+-----+------+--------+----------+---------------

In [29]:
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_41dc39073ea6


In [30]:
val accuracy = evaluator.evaluate(results)

accuracy: Double = 0.6163141993957704


In [31]:
val Precisionevaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("weightedPrecision")
val precision = Precisionevaluator.evaluate(results)

Precisionevaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_0087f50dfa29
precision: Double = 0.3798431923768494


In [32]:
val Recallevaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("weightedPrecision")
val Recall = Recallevaluator.evaluate(results)

Recallevaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_92c10ae8516f
Recall: Double = 0.3798431923768494


In [33]:
val F1scoreevaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("f1")
val F1Score = F1scoreevaluator.evaluate(results)

F1scoreevaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_65dafa756a04
F1Score: Double = 0.47001157636163415


In [34]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

val AreaROCEvaluator = new BinaryClassificationEvaluator()
  .setLabelCol("label")
  .setRawPredictionCol("rawPrediction")
  .setMetricName("areaUnderROC")
val AreaROC = AreaROCEvaluator.evaluate(results)

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
AreaROCEvaluator: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_c82e4a81ed8f
AreaROC: Double = 0.5
